In [ ]:
using Transformation, Plots, Logging
disable_logging(LogLevel(1000));
include("graphics.jl");

# Манипулятор PUMA
Манипулятор типа PUMA обладает шестью степенями подвижности и способен обеспечить шесть степеней свободы инструмента.

Для описания простейшей версии манипулятора кинематики PUMA достаточно задать длины четырех смещений вдоль локальных осей Z.

In [ ]:
l = [
    4, # Расстояние от основания до второй кинематической пары
    5, # Расстояние от второй кинематической пары до третьей
    4, # Расстояние от третьей кинематической пары до пятой
    1  # Расстояние от пятой кинематической пары до фланца
];

## Решение прямой задачи кинематики для манипулятора PUMA

Для того чтобы решить прямую задачу кинематики зададим положение всех кинематических пар

In [ ]:
function puma(l::Array{<:Real,1}, q::Array{<:Real,1})::Array{Transf,1}
    # Переведем градусы в радианы
    q = map(deg2rad, q)
    # Зададим нулевое смещение и поворот вокруг вертикальной оси на угол q[1]
    t0 = Transf(Vec(), Quat(q[1], Vec(0, 0, 1)))
    # Прибавим к нему смещение на длину l[1] и последующий поворот вокруг оси локальной X на угол q[2]
    t1 = t0 + Transf(Vec(0, 0, l[1]), Quat(q[2], Vec(1, 0, 0)))
    # Прибавим смещение на длину l[2] вдоль локальной оси Z и поворот вокруг локальной оси X на угол q[3]
    t2 = t1 + Transf(Vec(0, 0, l[2]), Quat(q[3], Vec(1, 0, 0)))
    # Прибавим смещение на длину l[3] и последовательные повороты вокруг локальных осей Z и X на углы q[4] и q[5]
    t3 = t2 + Transf(Vec(0, 0, l[3]), Quat(q[4], Vec(0, 0, 1)) * Quat(q[5], Vec(1, 0, 0)))#
    # Прибавим смещение на длину l[4] и последующий поворот вокруг оси Z на угол q[6]
    t4 = t3 + Transf(Vec(0, 0, l[4]), Quat(q[6], Vec(0, 0, 1)))
    [t0, t1, t2, t3, t4]
end;

## Пример решения прямой задачи кинематики

In [ ]:
qs = [0, 45, 90, 90, 90, 0]
qe = [120, -15, 120, 60, -90, 360]

pathX = []
pathY = []
pathZ = []
@gif for t = 0:.01:1
    # Подготовим график
    p = plot(xlims = (-6, 6), ylims = (-6, 6), zlims = (0, 12))
    
    # Зададим обобщенные координаты как промежуточное значение между начальным и конечным значениями
    q = qs + t * (qe - qs)
    
    # Расчитаем кинематическую цепь
    chain = puma(l, q)
    
    # Возьмем последнее положение из кинематической цепи
    lst = last(chain)
    push!(pathX, lst.v.x); push!(pathY, lst.v.y); push!(pathZ, lst.v.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    for joint in chain
        plotAxis!(p, joint, 0.5)
    end
    plot!(p, pathX, pathY, pathZ, color = :gray, label = "path")
end

## Решение обратной задачи кинематики для манипулятора PUMA

Для решения обратной задачи кинематики разобьем манипулятор на две части.
Первая часть будет отвечать за перемешение инструмента, вторая - за ориентирование.
К первой части относятся первые три подвижности, ко второй - последние три.

### Перемещение инструмента

Начнем с нахождения положения пятой кинематической пары.
Известно, что она смещена на $l_4$ вдоль локальной оси $Z_t$.
Тогда можно просто определить ее положение добавив вектор $(0, 0, -l_4)$ к системе координат, описывающей целевое положение `t`.

Кроме этого заметим, что робот приподнят на высоту $l_1$, для упрощения расчетов вычтем ее.

Теперь рассмотрим проекцию этой точки на плоскость $XY$.
Очевидно что угол от оси $X$ до этой проекции равен $\tan^{-1}(y, x)$.
Так как при $q_1 = 0$ робот направлен вдоль оси $Y$ вычтем недостающие $\frac{\pi}{2}$ радиан.
$$ q_1 = \tan^{-1}(y, x) - \frac{\pi}{2} $$

Рассмотрим плоскость образованную пятой кинематической парой и глобальной осью $Z$.
Плечо и предплечье робота ($l_2$ и $l_3$) образуют треугольник, лежащий в этой плоскости.
Тогда длина третьей стороны $d$ этого треугольника равна расстоянию от второй кинематической пары до пятой.
Угол при вершине этого треугольника $\beta$ можно определить по теореме косинусов.
$$ \beta = \cos^{-1}\left(\frac{l_2^2 + l_3^2 - d^2}{2 l_2 l_3}\right) $$

Так как $q_3$ расчитывается от продолжения звена, добавим $\pi$ к $b\eta$
$$ q_3 = \beta + \pi $$

При расчете $q_2$ заметим, что сторона $d$ треугольника приподнята на $\alpha_1 = \tan^{-1}(z, \sqrt{x^2 + y^2})$ относительно горизонтали.
Величина угла треугольника при второй кинематической паре равна (тоже по теореме косинусов) $\alpha_2 = \cos^{-1}\left(\frac{l_2^2 + d^2 - l_3^2}{2 l_2 d}\right)$.
Так как $q_2$ задано от продолжения звена, а $\alpha_1 + \alpha_2$ дают угол от горизонтали, вычтем из них $\frac{\pi}{2}$
$$q _2 = \alpha_1 + \alpha_2 - \frac{\pi}{2} $$

### Вращение инструмента

Для определения вращения инструмента сначала определим ориентацию робота после применения первых трех подвижностей.
$$ o =
\left[\cos\left(\frac{q_1}{2}\right], 0, 0, \sin\left(\frac{q_1}{2}\right)\right] \times
\left[\cos\left(\frac{q_2}{2}\right), \sin\left(\frac{q_2}{2}\right), 0, 0\right] \times
\left[\cos\left(\frac{q_3}{2}\right), \sin\left(\frac{q_3}{2}\right), 0, 0\right] $$

Отметим что ось вращения пятой кинематической пары перпендикулярна осям $Z_o$ четвертой кинематической пары $Z_t$ шестой кинематической пары.
Ось $Z$ шестой кинематической пары совпадает с осью $Z_t$ целевой ориентации.
Тогда ось $X_r$ пятой кинематической пары можно получить как векторное проихведение $Z_o$ и $Z_r$.
$$ X_r = Z_o \times{} Z_r $$

Тогда $q_4$ равно углу от оси $X_o$ третьей кинематической пары до оси $X_r$ пятой кинематической пары.
$$ q_4 = \angle{} X_o, X_t $$

$q_5$ равно углу от оси $Z_o$ четвертой кинематической пары до оси $Z_t$
$$ q_5 = \angle{} Z_o, Z_t $$

$q_6$ равно углу от оси $X_r$ пятой кинематической пары до оси $X_t$ целевой ориентации
$$ q_6 = \angle{} X_r, X_t $$

In [ ]:
function angleTo(a::Vec, b::Vec, n::Vec)::Real
    c = cross(a, b)
    d = dot(a, b)
    s = dot(c, n) > 0 ? 1 : -1
    s * atan(norm(c), d)
end

function wrap360(a::Real)::Real
    (a + 540) % 360 - 180
end

function ik(l::Array{<:Real,1}, t::Transf, i::Array{<:Real,1}=[1, 1, 1])::Array{<:Real,1}
    fifth = t + Vec(0, 0, -l[4])
    fifth = fifth - Vec(0, 0, l[1])
    q1 = atan(fifth.y, fifth.x) - pi + pi / 2 * i[1]
    d = norm(fith)
    q3 = i[1] * i[2] * acos((l[2]^2 + l[3]^2 - d^2)/(2 * l[2] * l[3])) + pi
    q2 = i[1] * (i[2] * acos((l[2]^2 + d^2 - l[3]^2)/(2 * l[2] * d)) + atan(fifth.z, sqrt(fifth.x^2 + fifth.y^2)) - pi / 2)
    
    o = Quat(q1, Vec(0, 0, 1)) * Quat(q2, Vec(1, 0, 0)) * Quat(q3, Vec(1, 0, 0))
    xo = o * Vec(1, 0, 0)
    zo = o * Vec(0, 0, 1)
    xt = t.q * Vec(1, 0, 0)
    zt = t.q * Vec(0, 0, 1)
    xr = cross(zo, zt)
    
    q4 = angleTo(xo, xr, zo) + pi / 2 - pi / 2 * i[3]
    q5 = angleTo(zo, zt, xr) * i[3]
    q6 = angleTo(xr, xt, zt) + pi / 2 - pi / 2 * i[3]
    q = map(rad2deg, [q1, q2, q3, q4, q5, q6])
    map(wrap360, q)
end;

## Начальное и конечное положения

In [ ]:
s = Transf(Vec(0, -6, 4), Quat(pi / 3, Vec(1, 0, 0)))
e = Transf(Vec(4, 0, 12), Quat(pi / 2, Vec(0, 1, 0)));

### Линейное движение

In [ ]:
pathX = []
pathY = []
pathZ = []
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        size = (500, 500)
    )
    # Определим промежуточное положение через линейные интерполяции вектора смещения и кватерниона ориентации
    target = Transf(
        s.v + t * (e.v - s.v),
        slerp(s.q, e.q, t),
    )
    # Воспользуемся решением обратной задачи кинематики
    q = ik(l, target, [1, 1, 1])
    chain = puma(l, q)
    lst = last(chain)
    push!(pathX, lst.v.x); push!(pathY, lst.v.y); push!(pathZ, lst.v.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, s); plotAxis!(p, e); plotAxis!(p, lst)
    plot!(p, pathX, pathY, pathZ, color = :gray, label = "path")
end

### Переброска

In [ ]:
pathX = []
pathY = []
pathZ = []

# Рещим обратную задачу кинематики для начального и конечного положения
qs = ik(l, s)
qe = ik(l, e)
@gif for t = 0:0.01:1
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        size = (500, 500)
    )
    # Найдем промежуточное значение обобщенных координат 
    q = qs + t * (qe - qs)
    chain = puma(l, q)
    lst = last(chain)
    push!(pathX, lst.v.x); push!(pathY, lst.v.y); push!(pathZ, lst.v.z)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, s); plotAxis!(p, e); plotAxis!(p, lst)
    plot!(p, pathX, pathY, pathZ, color = :gray, label = "path")
end

### Влияние флагов конфигурации

In [ ]:
# Перечислим все флаги используя коды Грея
flags = [
    [1, 1, 1],
    [-1, 1, 1],
    [-1, -1, 1],
    [-1, -1, -1],
    [-1, 1, -1],
    [1, 1, -1],
    [1, -1, -1],
    [1, -1, 1],
    [1, 1, 1]
]
# Зададим целевое положение
target = Transf(Vec(4, -3, 5), Quat(2pi / 3, Vec(1, 1, 1)))
@gif for i = 1:0.01:8.99
    p = plot(
        xlims = (-6, 6),
        ylims = (-6, 6),
        zlims = (0, 12),
        size = (500, 500)
    )
    
    flagsStart = flags[Int(floor(i))]
    flagsEnd = flags[Int(floor(i+1))]
    t = i % 1
    arg = 0.5 - cos(t * pi) / 2
    qStart = ik(l, target, flagsStart)
    qEnd = ik(l, target, flagsEnd)
    q = qStart + arg * (qEnd - qStart)
    
    chain = puma(l, q)
    lst = last(chain)
    plotChain!(p, chain)
    projectChain!(p, chain)
    plotAxis!(p, lst)
    plotAxis!(p, target)
end